In [7]:
from data.get_data import get_common_data
from model.LSTM_base_model import train_start_date,train_end_date
import  pandas as pd


#转换为df
df = pd.DataFrame(get_common_data('SHSE.510300',train_start_date,train_end_date,1,threshold=0.001).iloc[:,-1].rename('Y_pred'))

In [8]:
from model.LSTM_base_model import calculate_performance_score

calculate_performance_score(df['Y_pred'], df['Y_pred'])

2.0

In [1]:
from LSTM_base_model import  lstm_predict,best_lstm_model,val_start_date,val_end_date

model_dict = best_lstm_model(T=3)
model_dict

{'T': 3,
 'window_size': 120,
 'hidden_dim': 30,
 'num_layers': 3,
 'model_path': 'lstm_models_7\\best_lstm_model_T3_window120_hidden30_layers3.pth'}

In [3]:
result = lstm_predict(model_dict,start_date=val_start_date,end_date=val_end_date)
result.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 378 entries, 2020-07-09 to 2021-12-20
Freq: B
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Y_true       378 non-null    int64  
 1   Y_pred       378 non-null    int64  
 2   Prob_down    378 non-null    float32
 3   Prob_stable  378 non-null    float32
 4   Prob_up      378 non-null    float32
dtypes: float32(3), int64(2)
memory usage: 13.3 KB


In [5]:
# 查看Y_true和Y_pred的分布
result['Y_true'].value_counts()
# result['Y_pred'].value_counts()

2    166
0    155
1     57
Name: Y_true, dtype: int64

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
# 假设y_true是真实标签，y_pred是预测标签
y_true = result['Y_true']
y_pred = result['Y_pred']


# 计算混淆矩阵
conf_matrix = confusion_matrix(y_true, y_pred)
# 混淆矩阵标签优化
conf_matrix = pd.DataFrame(conf_matrix, columns=['Predicted 0', 'Predicted 1', 'Predicted 2'], index=['Actual 0', 'Actual 1', 'Actual 2'])

# 计算预测为0的召回率
recall_0 = conf_matrix.iloc[0, 0] / conf_matrix.iloc[0, :].sum()

#计算预测为2的召回率
recall_2 = conf_matrix.iloc[2, 2] / conf_matrix.iloc[2, :].sum()

# 计算应该为2，但是预测为0；应该为0，但是预测为2的概率
fpr_0 = conf_matrix.iloc[0, 2] / conf_matrix.iloc[0, :].sum()
fpr_2 = conf_matrix.iloc[2, 0] / conf_matrix.iloc[2, :].sum()

# 用一个指标融合这四个指标，其中召回率越高越好，fpr越低越好
score = recall_0 + recall_2 - fpr_0 - fpr_2

score

NameError: name 'result' is not defined

In [6]:

# 计算准确率
accuracy = accuracy_score(y_true, y_pred)

# 计算精确率
precision = precision_score(y_true, y_pred)

# 计算对于0或2的平均召回率
recall = recall_score(y_true, y_pred, average='macro')




Accuracy: 0.6402116402116402
Confusion Matrix:
 [[116   0  39]
 [ 28   0  29]
 [ 40   0 126]]
Precision: 0.4266397728970566
Recall: 0.5024744137841689
F1 Score: 0.46145526057030484


E:\anaconda\envs\pytor\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
import pandas as pd
import os
import torch
from model.LSTM_base_model import prepare_data, LSTMModel

symbol = 'SHSE.510300'
start_date = val_start_date
end_date = val_end_date
X_test, Y_true = prepare_data(symbol, start_date, end_date, model_dict['T'], model_dict['window_size'])
# 检查如果GPU没有使用成功，raise error
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    raise ValueError('GPU not available')
try:
    model_path = model_dict['model_path']
except KeyError:
    model_path = model_dict['best_model_path']
model = LSTMModel(X_test.shape[2], int(model_dict['hidden_dim']), model_dict['num_layers'], device).to(device)
model.load_state_dict(torch.load(os.path.join('../model/', model_path)))
model.eval()
with torch.no_grad():
    outputs = model(X_test.float().to(device))
    outputs

outputs

tensor([[-0.3946, -0.6753,  1.0613],
        [ 0.6718, -0.4055,  0.0966],
        [-0.0296, -0.5372,  0.7358],
        ...,
        [ 0.9426, -0.6852,  0.1544],
        [ 1.0646, -0.6829,  0.0845],
        [ 1.1624, -0.6667,  0.0036]], device='cuda:0')

In [6]:
len(outputs),len(Y_true)

(378, 378)

In [8]:
len(outputs.cpu().detach().numpy().flatten())

1134

In [10]:
import torch.nn.functional as F
probabilities = F.softmax(outputs, dim=1).cpu().detach().numpy()
probabilities

array([[0.16546279, 0.1249718 , 0.7095654 ],
       [0.52546036, 0.178928  , 0.29561165],
       [0.26654235, 0.16044284, 0.5730148 ],
       ...,
       [0.60568404, 0.11892876, 0.2753872 ],
       [0.6453713 , 0.11242756, 0.24220113],
       [0.67822105, 0.10889746, 0.21288155]], dtype=float32)

In [1]:
from model.LSTM_base_model import best_lstm_model

model_dict = best_lstm_model(T=3)
model_dict

{'T': 3,
 'window_size': 120,
 'hidden_dim': 30,
 'num_layers': 3,
 'model_path': 'lstm_models_7\\best_lstm_model_T3_window120_hidden30_layers3.pth',
 'val_my_score': (3, 1.0148464827050137)}

# XGB test

In [2]:
from model.my_xgboost import lstm_to_xgboost, train_xgboost_classify
import xgboost as xgb
from model.LSTM_base_model import val_start_date, val_end_date,config_id
import  os

target_T = 3
modelDirectory = f'../model/xgb_models_{config_id}'

start_date = val_start_date
end_date = val_end_date
X, Y_true = lstm_to_xgboost(start_date, end_date, target_T)
# 加载模型
bst = xgb.Booster()

if os.path.exists(f'{modelDirectory}/xgb_model_{target_T}.json'):
    print(f'Loading xgb_model_{target_T}.json')
    bst.load_model(f'{modelDirectory}/xgb_model_{target_T}.json')
else:
    print(f'Training xgb_model_{target_T}.json')
    train_xgboost_classify(target_T)
    bst.load_model(f'{modelDirectory}/xgb_model_{target_T}.json')
# 将数据转换为DMatrix对象，XGBoost专用的数据结构
dtest = xgb.DMatrix(X)
# 预测
Y_pred = bst.predict(dtest)
Y_pred

Loading xgb_model_3.json


array([[5.5825565e-02, 8.8440783e-02, 8.5573363e-01],
       [7.6943469e-01, 2.0223601e-01, 2.8329294e-02],
       [7.5407684e-01, 2.4545860e-01, 4.6455438e-04],
       ...,
       [8.5120720e-01, 9.0110444e-02, 5.8682390e-02],
       [8.5120720e-01, 9.0110444e-02, 5.8682390e-02],
       [8.5120720e-01, 9.0110444e-02, 5.8682390e-02]], dtype=float32)

In [2]:
from model.my_xgboost import train_xgboost_classify

train_xgboost_classify(15)

Running best_lstms: 100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


[0]	eval-mlogloss:0.93159
[1]	eval-mlogloss:0.84016
[2]	eval-mlogloss:0.78577
[3]	eval-mlogloss:0.75297
[4]	eval-mlogloss:0.73436
[5]	eval-mlogloss:0.72374
[6]	eval-mlogloss:0.72085
[7]	eval-mlogloss:0.72046
[8]	eval-mlogloss:0.72106
[9]	eval-mlogloss:0.72093
[10]	eval-mlogloss:0.72410
[11]	eval-mlogloss:0.72944
[12]	eval-mlogloss:0.73426
[13]	eval-mlogloss:0.74183
[14]	eval-mlogloss:0.74085
[15]	eval-mlogloss:0.74401
[16]	eval-mlogloss:0.74326
[17]	eval-mlogloss:0.74306
[18]	eval-mlogloss:0.74945
[19]	eval-mlogloss:0.75472
[20]	eval-mlogloss:0.75798
[21]	eval-mlogloss:0.75915
[22]	eval-mlogloss:0.75931
[23]	eval-mlogloss:0.76152
[24]	eval-mlogloss:0.76350
[25]	eval-mlogloss:0.76579
[26]	eval-mlogloss:0.77159
[27]	eval-mlogloss:0.77538
[28]	eval-mlogloss:0.77975
[29]	eval-mlogloss:0.78135
[30]	eval-mlogloss:0.78366
[31]	eval-mlogloss:0.78561
[32]	eval-mlogloss:0.78737
[33]	eval-mlogloss:0.78802
[34]	eval-mlogloss:0.79329
[35]	eval-mlogloss:0.79657
[36]	eval-mlogloss:0.79566
[37]	eval-m

<Figure size 1000x800 with 0 Axes>

# LSTM 测试

In [1]:
from model.LSTM_base_model import lstm_predict, best_lstm_model, val_start_date, val_end_date,prepare_data
from data.get_data import get_common_data

In [5]:
from data.get_data import my_get_next_n_trading_date
set = 'val'
my_get_next_n_trading_date(date=eval(f'{set}_start_date'), counts=1)

'2020-07-10'

In [4]:
data = get_common_data('SHSE.510300', val_start_date, val_end_date, 1, threshold=0.001)
data

,open,high,low,close,volume,amount,pre_close,MA_5,MA_10,MA_20,...,ppi,shibor_on,shibor_1w,shibor_2w,shibor_1m,shibor_3m,shibor_6m,shibor_9m,shibor_1y,1_return_to_trend
bob,,,,,,,,,,,,,,,,,,,,,
2020-07-09,4.796,4.884,4.760,4.872,642434176,3104867392.0,4.798,4.7294,4.4759,4.26030,...,-3.1,2.189,2.192,2.048,2.013,2.122,2.222,2.353,2.420,0
2020-07-10,4.838,4.857,4.763,4.785,564416768,2719824192.0,4.872,4.7950,4.5380,4.29755,...,-3.1,2.197,2.203,1.957,2.035,2.143,2.241,2.367,2.433,2
2020-07-13,4.782,4.927,4.777,4.885,824378256,4008141936.0,4.785,4.8152,4.6137,4.34165,...,-3.1,2.128,2.201,2.216,2.057,2.163,2.264,2.385,2.455,0
2020-07-14,4.877,4.899,4.765,4.842,737641424,3569719088.0,4.885,4.8364,4.6801,4.38310,...,-3.1,1.670,2.148,2.094,2.077,2.192,2.292,2.404,2.480,0
2020-07-15,4.860,4.895,4.765,4.784,662402944,3200591200.0,4.842,4.8336,4.7313,4.42450,...,-3.1,2.022,2.150,2.113,2.100,2.241,2.334,2.438,2.516,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-18,4.832,4.897,4.815,4.881,624167056,3038466592.0,4.835,4.8534,4.8775,4.93675,...,12.9,1.921,2.088,2.209,2.427,2.493,2.548,2.640,2.706,0
2022-01-19,4.815,4.839,4.756,4.780,522439264,2501399280.0,4.806,4.8256,4.8612,4.92680,...,12.9,2.035,2.127,2.405,2.422,2.483,2.534,2.630,2.696,2
2022-01-20,4.775,4.847,4.774,4.818,541278496,2608626688.0,4.780,4.8216,4.8543,4.91860,...,12.9,2.035,2.113,2.431,2.418,2.471,2.526,2.623,2.686,0


In [6]:
data.index

DatetimeIndex(['2020-07-09', '2020-07-10', '2020-07-13', '2020-07-14',
               '2020-07-15', '2020-07-16', '2020-07-17', '2020-07-20',
               '2020-07-21', '2020-07-22',
               ...
               '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14',
               '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20',
               '2022-01-21', '2022-01-24'],
              dtype='datetime64[ns]', name='bob', length=378, freq=None)

In [2]:
import torch.nn.functional as F
symbol = 'SHSE.510300'
start_date = val_start_date
end_date = val_end_date
# model_dict = best_lstm_model(T=3)
dateIndex, X_test, Y_true = prepare_data(symbol, start_date, end_date, 3, 40)
len(dateIndex),len(Y_true)

(378, 378)

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    raise ValueError('GPU not available')
try:
    model_path = model_dict['model_path']
except KeyError:
    model_path = model_dict['best_model_path']

model = LSTMModel(X_test.shape[2], int(model_dict['hidden_dim']), model_dict['num_layers'], device).to(device)
model.load_state_dict(torch.load(os.path.join('../model/', model_path)))
model.eval()

with torch.no_grad():
    outputs = model(X_test.float().to(device))
    # 将logits转换为概率
    probabilities = F.softmax(outputs, dim=1).cpu().detach().numpy()

Y_pred = probabilities.argmax(axis=1)  # 获取最大概率的索引，用于分类
# 处理真实值与预测概率并合并为DataFrame
Y_true = Y_true.flatten()
Y_true = pd.DataFrame(Y_true, columns=['Y_true'])
Y_pred = pd.DataFrame(Y_pred, columns=['Y_pred'])
probabilities_df = pd.DataFrame(probabilities, columns=['Prob_down', 'Prob_stable', 'Prob_up'])
result = pd.concat([Y_true, Y_pred, probabilities_df], axis=1)
result

NameError: name 'torch' is not defined

In [9]:

model_dict = best_lstm_model(T=1)
result = lstm_predict(model_dict, start_date=val_start_date, end_date=val_end_date)
result

,Y_true,Y_pred,Prob_down,Prob_stable,Prob_up
date,,,,,
2020-07-09,0,2,0.44450995,0.08033007,0.47515997
2020-07-10,2,2,0.44435701,0.07839045,0.47725260
2020-07-13,0,2,0.44440416,0.07888719,0.47670865
2020-07-14,0,2,0.44438717,0.07865525,0.47695759
2020-07-15,0,2,0.44439158,0.07864609,0.47696236
...,...,...,...,...,...
2022-01-18,0,2,0.44440025,0.07866643,0.47693336
2022-01-19,2,2,0.44440117,0.07867598,0.47692287
2022-01-20,0,2,0.44440156,0.07867481,0.47692364


In [11]:
temp = result['Y_pred'][1]
temp

2

In [12]:
set = 'val'
backtest_start_time = str(eval(f'{set}_start_date')) + ' 08:00:00'
backtest_end_time = eval(f'{set}_end_date')+ ' 16:00:00'
backtest_end_time

'2022-01-24 16:00:00'